In [ ]:
from pynq import Overlay
import numpy as np

In [ ]:
ol = Overlay("dSB_2048_16.bit")

In [ ]:
ol.ip_dict.keys()

In [ ]:
dSB = ol.dSB_0

In [ ]:
dSB.register_map

In [ ]:
N = 2048

In [ ]:
A = (-1)*np.ones((N, N)) + np.eye(N)

In [ ]:
A

In [ ]:
J = A

In [ ]:
#sum_J_power = 0
#for i in range(N):
#    for j in range(N):
#        sum_J_power += J[i][j] * J[i][j]

In [ ]:
#xi = 0.5 * np.sqrt(N - 1) / np.sqrt(sum_J_power)

In [ ]:
#xi

In [ ]:
def ap_fixed_to_int(binary_str):
    """
    Convert a 16-bit two's complement fixed-point number to an integer.
    
    Args:
    binary_str (str): A 16-bit binary string
    
    Returns:
    float: The decimal integer value
    """
    # Validate input
    if len(binary_str) != 16:
        raise ValueError("Input must be a 16-bit binary string")
    
    # Check if it's a negative number (first bit is 1)
    if binary_str[0] == '1':
        # For negative numbers, invert the bits and add 1
        inverted = ''.join('1' if bit == '0' else '0' for bit in binary_str)
        return -(int(inverted, 2) + 1)
    else:
        # For positive numbers, simply convert to decimal
        return int(binary_str, 2)

def detailed_ap_fixed_conversion(binary_str):
    """
    Provide detailed conversion of a 16-bit two's complement fixed-point number.
    
    Args:
    binary_str (str): A 16-bit binary string
    
    Returns:
    dict: Detailed conversion information
    """
    # Validate input
    if len(binary_str) != 16:
        raise ValueError("Input must be a 16-bit binary string")
    
    # Breakdown the bits
    sign_bit = binary_str[0]
    integer_bits = binary_str[1:4]  # 3 magnitude bits for integer part
    fractional_bits = binary_str[4:]  # 12 fractional bits
    
    # Convert to integer value
    if sign_bit == '1':
        # Negative number: invert and add 1
        inverted = ''.join('1' if bit == '0' else '0' for bit in binary_str)
        int_value = -(int(inverted, 2) + 1)
    else:
        int_value = int(binary_str, 2)
    
    return {
        'binary': binary_str,
        'sign_bit': sign_bit,
        'integer_bits': integer_bits,
        'fractional_bits': fractional_bits,
        'integer_value': int_value,
        'decimal_representation': int_value / (2 ** 12)  # Divide by 2^12 due to 12 fractional bits
    }



In [ ]:
def get_result_x(r_x):
    count = 0
    for j in range(N):
        count = j;
        c_a = count%2
        if c_a == 1:
            tmpt = dSB.read(0x1000+4*int(count/2))
            cyc = bin(tmpt)[2:34]
            #print(cyc)
            cyc = cyc.zfill(32)
            #print(cyc)
            a = ap_fixed_to_int(cyc[0:16])
            #print(len(cyc[18:34]))
            #print(cyc)
            #print(cyc[18:34])
            b = ap_fixed_to_int(cyc[16:32])
            #print(a>>12, b>>12)
            #r_x[j-1] = np.sign(a)#>>12
            #r_x[j] = np.sign(b)#>>12
            r_x[j-1] = a/(2**14)
            r_x[j] = b/(2**14)
rx = np.zeros(N)

In [ ]:
x = 0.02 * (np.random.rand(N) - 0.5)
y = 0.02 * (np.random.rand(N) - 0.5)
for i in range(N):
    x[i] *= (2**14)
    y[i] *= (2**14)
x=x.astype(np.int16)
y=y.astype(np.int16)

In [ ]:
dt = 0.01*(2**14)
print(dt)
xi = 0.015625*(2**14)
print(xi)
tmpt = np.zeros((2),dtype="int16")
tmpt[0] = np.int16(dt)
dSB.write(0x18, bytes(tmpt))
tmpt[0] = np.int16(xi)
dSB.write(0x10, bytes(tmpt))


count = 0
for j in range(N):
    count = j;
    c_a = count%2
    if c_a == 0:
        tmpt = np.zeros((2),dtype="int16")
    tmpt[c_a] = x[j]
    if c_a == 1:
        val = bytes(tmpt)
        dSB.write(0x1000+4*int(count/2), val)

count = 0
for j in range(N):
    count = j;
    c_a = count%2
    if c_a == 0:
        tmpt = np.zeros((2),dtype="int16")
    tmpt[c_a] = y[j]
    if c_a == 1:
        val = bytes(tmpt)
        dSB.write(0x2000+4*int(count/2), val)


In [ ]:
%%time
dSB.write(0,1)

finished = dSB.read(0)
while(finished!=14):
    finished = dSB.read(0)

In [ ]:
get_result_x(rx)
np.sum(rx>0)

In [ ]:
rx